In [3]:
import pandas as pd
import numpy as np
import re

from google.colab import drive
drive.mount('/content/drive')

import os

#path = '/content/drive/My Drive/Colab Notebooks/Mini-Projet/sensor_data_processed_normalizing_the_label_with_hot_encoding_index.csv'

path = '/content/drive/My Drive/Colab Notebooks/Mini-Projet/sensor_data_processed_filling_missing_values.csv'

all_data = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
all_data.insert(0,'Timestamps',all_data.index)

In [6]:
all_data['Timestamps'] = pd.to_datetime(all_data['Timestamps'])

In [7]:
all_data['Timestamps'] = all_data['Timestamps'].apply(lambda x: (x - all_data['Timestamps'].min()) / (all_data['Timestamps'].max() - all_data['Timestamps'].min()))

# Normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, OneHotEncoder, MultiLabelBinarizer

In [9]:
# Get the list of binary column names
binary_cols = []
for col in all_data.columns:
    if all_data[col].nunique() == 2:  # Check if the column has only two unique values
        binary_cols.append(col)

binary_cols

['bedroom_switch_middle_left',
 'bedroom_switch_middle_right',
 'livingroom_heater2_base_setpoint',
 'kitchen_washingmachine_partial_energy',
 'walkway_switch2_bottom_left',
 'livingroom_shutter5',
 'livingroom_shutter4',
 'livingroom_switch2_top_left',
 'livingroom_tv_status',
 'bedroom_closet_door',
 'bathroom_switch_bottom_right',
 'bedroom_light4',
 'office_heater_effective_mode',
 'kitchen_cupboard3',
 'toilet_switch_left',
 'entrance_heater_base_setpoint',
 'bedroom_light2',
 'kitchen_fridge_door',
 'livingroom_presence_table',
 'bathroom_switch_bottom_left',
 'staircase_light',
 'bathroom_switch_top_right',
 'bedroom_switch_bottom_left',
 'livingroom_presence_couch',
 'bedroom_presence',
 'toilet_switch_right',
 'livingroom_heater1_effective_mode',
 'bedroom_shutter1',
 'walkway_switch1_bottom_left',
 'bedroom_drawer1',
 'office_switch_right',
 'walkway_switch1_bottom_right',
 'livingroom_light1',
 'entrance_switch_left',
 'bathroom_sink_hotwater_instantaneous',
 'staircase_swit

In [10]:
# Get the list of columns with multiple categories
multi_cols = []
for col in all_data.columns:
    if all_data[col].dtype == "object" and all_data[col].nunique() > 2:  # Check if the column is categorical and has more than two unique values
        multi_cols.append(col)
  
multi_cols

['global_wind_direction_ext',
 'global_pressure_trend_ext',
 'global_condition_ext',
 'global_commonID_ext',
 'label']

In [11]:
# select columns with numeric data types
numerical_cols = all_data.select_dtypes(include=np.number).columns.tolist()

# print the list of numerical columns
print(numerical_cols)

['Timestamps', 'entrance_heater_effective_setpoint', 'kitchen_noise', 'livingroom_couch_plug_consumption', 'global_snow_ext', 'bedroom_heater1_effective_setpoint', 'global_lighting_power', 'livingroom_heater2_base_setpoint', 'kitchen_dishwasher_current', 'toilet_coldwater_total', 'kitchen_hood_voltage', 'kitchen_washingmachine_partial_energy', 'livingroom_shutter5', 'kitchen_hood_total_energy', 'bathroom_heater_command', 'walkway_noise', 'entrance_noise', 'livingroom_shutter4', 'global_rain_ext', 'global_wind_speed_ext', 'global_waterheater_total_energy', 'global_lighting_current', 'livingroom_heater2_temperature', 'office_tv_plug_consumption', 'kitchen_fridge_power', 'bathroom_luminosity', 'livingroom_tv_plug_consumption', 'office_AC_setpoint', 'bedroom_light4', 'office_heater_effective_mode', 'global_lighting_total_energy', 'entrance_heater_base_setpoint', 'office_heater_command', 'bedroom_light2', 'livingroom_table_luminosity', 'global_voltage', 'bedroom_temperature', 'bathroom_sink

In [12]:
all_data['label']

0         START:Entrance|Entering
1         START:Entrance|Entering
2         START:Entrance|Entering
3         START:Entrance|Entering
4         START:Entrance|Entering
                   ...           
383709     START:Entrance|Leaving
383710     START:Entrance|Leaving
383711     START:Entrance|Leaving
383712     START:Entrance|Leaving
383713      STOP:Entrance|Leaving
Name: label, Length: 383714, dtype: object

In [13]:
# Select rows where the 'label' column starts with 'stop'
mask = all_data['label'].str.startswith('STOP')

# Delete the selected rows
all_data = all_data[~mask]

In [14]:
all_data['label']

0         START:Entrance|Entering
1         START:Entrance|Entering
2         START:Entrance|Entering
3         START:Entrance|Entering
4         START:Entrance|Entering
                   ...           
383708     START:Entrance|Leaving
383709     START:Entrance|Leaving
383710     START:Entrance|Leaving
383711     START:Entrance|Leaving
383712     START:Entrance|Leaving
Name: label, Length: 378171, dtype: object

In [15]:
# we are replacing the Start:activity by activity,
all_data['label'] = all_data['label'].str.replace('START:', '', flags=re.IGNORECASE)

In [16]:
all_data['label']

0         Entrance|Entering
1         Entrance|Entering
2         Entrance|Entering
3         Entrance|Entering
4         Entrance|Entering
                ...        
383708     Entrance|Leaving
383709     Entrance|Leaving
383710     Entrance|Leaving
383711     Entrance|Leaving
383712     Entrance|Leaving
Name: label, Length: 378171, dtype: object

## Normalizing numerical values using Min-Max


In [17]:
# Apply min-max scaling
scaler = MinMaxScaler()
for col in numerical_cols:
    all_data[col] = scaler.fit_transform(all_data[[col]])

## Normalizing categorical values using LabelBinarizer





In [18]:
# Apply LabelBinarizer to binary columns only
lb = LabelBinarizer()
for col in binary_cols:
    all_data[col] = lb.fit_transform(all_data[[col]])

## Normalizing categorical values using One-hot encoding


In [19]:
# Apply one-hot encoding to binary columns only
encoder = OneHotEncoder()

for col in multi_cols:
    print(col, ' ',all_data[col].unique())


global_wind_direction_ext   ['ENE' 'SSE' 'S' 'E' 'SSW' 'SW' 'WSW' 'W' 'NNW' 'WNW' 'N' 'NE' 'ESE' 'SE'
 'NW' 'NNE']
global_pressure_trend_ext   ['down' 'up' 'equal']
global_condition_ext   ['peu nuageux' 'Couvert' 'légères pluies' 'brume' 'brouillard'
 'partiellement ensoleillé' 'nuageux' 'ensoleillé']
global_commonID_ext   ['partly-cloudy-day' 'cloudy' 'few-showers' 'fog' 'mostly-cloudy-day'
 'sunny']
label   ['Entrance|Entering' 'Staircase|Going_up' 'Bathroom|Showering'
 'Bathroom|Using_the_sink' 'Staircase|Going_down'
 'Living_room|Watching_TV' 'Toilet|Using_the_toilet' 'Office|Computing'
 'Kitchen|Preparing' 'Kitchen|Cooking' 'Living_room|Eating'
 'Kitchen|Washing_the_dishes' 'Living_room|Cleaning'
 'Living_room|Computing' 'Bedroom|Dressing' 'Bedroom|Reading'
 'Bedroom|Napping' 'Bathroom|Using_the_toilet' 'Office|Watching_TV'
 'Entrance|Leaving' 'Kitchen|Cleaning' 'Bathroom|Cleaning'
 'Bedroom|Cleaning' 'Office|Cleaning']


In [20]:
all_data

,Timestamps,bedroom_switch_middle_left,entrance_heater_effective_setpoint,bedroom_switch_middle_right,kitchen_noise,livingroom_couch_plug_consumption,global_snow_ext,bedroom_heater1_effective_setpoint,global_lighting_power,livingroom_heater2_base_setpoint,...,global_waterheater_voltage,kitchen_cupboard4,bathroom_sink_coldwater_instantaneous,bedroom_noise,kitchen_humidity,kitchen_oven_power,global_commonID_ext,kitchen_presence,livingroom_luminosity,label
0,0.000000,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0.000000,1,0.0,0.00000,0.000,0.0,partly-cloudy-day,1,0.000000,Entrance|Entering
1,0.000003,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0.000000,1,0.0,0.00000,0.000,0.0,partly-cloudy-day,1,0.000000,Entrance|Entering
2,0.000005,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0.000000,1,0.0,0.00000,0.000,0.0,partly-cloudy-day,1,0.000000,Entrance|Entering
3,0.000008,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0.000000,1,0.0,0.00000,0.000,0.0,partly-cloudy-day,1,0.000000,Entrance|Entering
4,0.000010,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0.000000,1,0.0,0.00000,0.000,0.0,partly-cloudy-day,1,0.000000,Entrance|Entering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383708,0.999990,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0.989019,0,0.0,0.07171,0.875,0.0,cloudy,0,0.011103,Entrance|Leaving
383709,0.999992,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0.989019,0,0.0,0.07171,0.875,0.0,cloudy,0,0.011103,Entrance|Leaving
383710,0.999995,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0.989019,0,0.0,0.07171,0.875,0.0,cloudy,0,0.011103,Entrance|Leaving
383711,0.999997,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0.989019,0,0.0,0.07171,0.875,0.0,cloudy,0,0.011103,Entrance|Leaving


In [21]:
for col in multi_cols:
  # one-hot encode the 'label' column
  one_hot_df = pd.get_dummies(all_data[col])
  # concatenate the original dataframe with the one-hot encoded dataframe
  all_data = pd.concat([all_data, one_hot_df], axis=1)
  # drop the original 'label' column
  all_data.drop(col, axis=1, inplace=True)

In [22]:
all_data

,Timestamps,bedroom_switch_middle_left,entrance_heater_effective_setpoint,bedroom_switch_middle_right,kitchen_noise,livingroom_couch_plug_consumption,global_snow_ext,bedroom_heater1_effective_setpoint,global_lighting_power,livingroom_heater2_base_setpoint,...,Living_room|Cleaning,Living_room|Computing,Living_room|Eating,Living_room|Watching_TV,Office|Cleaning,Office|Computing,Office|Watching_TV,Staircase|Going_down,Staircase|Going_up,Toilet|Using_the_toilet
0,0.000000,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000003,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000005,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
3,0.000008,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000010,1,0.0,1,0.000000,0.0,0.0,0.0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383708,0.999990,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0,0,0,0,0,0,0,0,0,0
383709,0.999992,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0,0,0,0,0,0,0,0,0,0
383710,0.999995,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0,0,0,0,0,0,0,0,0,0
383711,0.999997,0,1.0,0,0.004371,0.0,0.0,1.0,0.124088,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# write the DataFrame to a CSV file (Normalizing the label with Hot encoding)
all_data.to_csv('sensor_data_processed_normalizing_the_label_with_hot_encoding.csv', index=False)

In [54]:
all_data = pd.read_csv('sensor_data_processed_normalizing_the_label_with_hot_encoding.csv')

In [55]:
X = all_data[['bedroom_switch_middle_left',
 'entrance_heater_effective_setpoint',
 'bedroom_switch_middle_right',
 'kitchen_noise',
 'livingroom_couch_plug_consumption',
 'global_snow_ext',
 'bedroom_heater1_effective_setpoint',
 'global_lighting_power',
 'livingroom_heater2_base_setpoint',
 'kitchen_dishwasher_current',
 'toilet_coldwater_total',
 'kitchen_hood_voltage',
 'kitchen_washingmachine_partial_energy',
 'walkway_switch2_bottom_left',
 'livingroom_shutter5',
 'kitchen_hood_total_energy',
 'bathroom_heater_command',
 'walkway_noise',
 'entrance_noise',
 'livingroom_shutter4',
 'livingroom_switch2_top_left',
 'global_rain_ext',
 'global_wind_speed_ext',
 'global_waterheater_total_energy',
 'global_lighting_current',
 'livingroom_heater2_temperature',
 'livingroom_tv_status',
 'office_tv_plug_consumption',
 'kitchen_fridge_power',
 'bedroom_closet_door',
 'bathroom_luminosity',
 'livingroom_tv_plug_consumption',
 'office_AC_setpoint',
 'bathroom_switch_bottom_right',
 'bedroom_light4',
 'office_heater_effective_mode',
 'global_lighting_total_energy',
 'kitchen_cupboard3',
 'toilet_switch_left',
 'entrance_heater_base_setpoint',
 'office_heater_command',
 'bedroom_light2',
 'livingroom_table_luminosity',
 'global_voltage',
 'bedroom_temperature',
 'kitchen_fridge_door',
 'bathroom_sink_coldwater_total',
 'global_lighting_partial_energy',
 'livingroom_couch_noise',
 'global_temperature_feel_ext',
 'livingroom_humidity',
 'kitchen_oven_current',
 'livingroom_presence_table',
 'bathroom_switch_bottom_left',
 'livingroom_heater1_command',
 'staircase_light',
 'office_noise',
 'kitchen_sink_hotwater_total',
 'bathroom_switch_top_right',
 'bedroom_switch_bottom_left',
 'livingroom_presence_couch',
 'livingroom_heater2_effective_setpoint',
 'bedroom_presence',
 'toilet_switch_right',
 'bedroom_humidity',
 'livingroom_heater1_effective_mode',
 'bedroom_heater2_effective_setpoint',
 'livingroom_heater2_command',
 'bedroom_shutter1',
 'global_active_power',
 'walkway_switch1_bottom_left',
 'kitchen_hood_current',
 'entrance_heater_temperature',
 'bedroom_CO2',
 'livingroom_table_plug_consumption',
 'bathroom_CO2',
 'bathroom_temperature',
 'bedroom_drawer1',
 'office_switch_right',
 'global_heaters_temperature',
 'global_waterheater_current',
 'kitchen_dishwasher_voltage',
 'global_shutters_voltage',
 'walkway_switch1_bottom_right',
 'livingroom_heater1_effective_setpoint',
 'livingroom_light1',
 'bathroom_heater_temperature',
 'kitchen_cooktop_current',
 'entrance_switch_left',
 'toilet_coldwater_instantaneous',
 'kitchen_oven_voltage',
 'bathroom_sink_hotwater_instantaneous',
 'staircase_switch_right',
 'kitchen_sink_coldwater_instantaneous',
 'livingroom_heater2_effective_mode',
 'office_tv_status',
 'global_shutters_current',
 'kitchen_fridge_current',
 'kitchen_washingmachine_total_energy',
 'bedroom_shutter2',
 'bathroom_light2',
 'global_waterheater_partial_energy',
 'global_frequency',
 'kitchen_switch_bottom_right',
 'bedroom_door',
 'kitchen_luminosity',
 'bedroom_heater2_command',
 'entrance_heater_command',
 'global_pressure_ext',
 'kitchen_oven_partial_energy',
 'global_shutters_power',
 'office_luminosity',
 'kitchen_cooktop_total_energy',
 'kitchen_washingmachine_voltage',
 'bathroom_light1',
 'bedroom_luminosity',
 'office_presence',
 'bathroom_heater_effective_setpoint',
 'bathroom_shower_hotwater_instantaneous',
 'livingroom_switch1_top_left',
 'bathroom_humidity',
 'kitchen_cupboard5',
 'bathroom_shower_coldwater_total',
 'bathroom_shower_coldwater_instantaneous',
 'office_window',
 'kitchen_switch_bottom_left',
 'kitchen_dishwasher_partial_energy',
 'kitchen_fridge_voltage',
 'office_heater_effective_setpoint',
 'office_heater_temperature',
 'livingroom_window1',
 'bathroom_sink_hotwater_total',
 'kitchen_cooktop_voltage',
 'kitchen_hood_power',
 'kitchen_sink_hotwater_instantaneous',
 'global_current',
 'global_condition_id_ext',
 'livingroom_switch1_top_right',
 'office_AC_mode',
 'bedroom_heater2_effective_mode',
 'walkway_light',
 'bathroom_door',
 'global_clouds_ext',
 'global_shutters_total_energy',
 'bedroom_heater1_effective_mode',
 'bedroom_light1',
 'kitchen_cupboard1',
 'livingroom_temperature',
 'livingroom_CO2',
 'global_shutters_partial_energy',
 'bathroom_shower_hotwater_total',
 'kitchen_switch_top_right',
 'kitchen_cooktop_partial_energy',
 'global_waterheater_power',
 'kitchen_sink_coldwater_total',
 'kitchen_oven_total_energy',
 'global_power_factor',
 'kitchen_washingmachine_current',
 'global_coldwater_instantaneous',
 'office_desk_plug_consumption',
 'livingroom_switch1_bottom_left',
 'entrance_door',
 'global_active_energy',
 'global_coldwater_total',
 'bathroom_heater_effective_mode',
 'office_light',
 'walkway_switch1_top_right',
 'global_humidity_ext',
 'livingroom_AC_setpoint',
 'walkway_switch2_top_right',
 'bathroom_switch_top_left',
 'kitchen_cupboard2',
 'office_heater_base_setpoint',
 'bedroom_light3',
 'kitchen_dishwasher_total_energy',
 'livingroom_shutter3',
 'kitchen_light2',
 'kitchen_cooktop_power',
 'office_switch_left',
 'toilet_light',
 'livingroom_shutter2',
 'bathroom_presence',
 'livingroom_light2',
 'bedroom_switch_top_right',
 'livingroom_heater1_temperature',
 'bedroom_switch_top_left',
 'walkway_switch1_top_left',
 'livingroom_switch2_top_right',
 'walkway_switch2_bottom_right',
 'kitchen_dishwasher_power',
 'global_heaters_setpoint',
 'bedroom_AC_setpoint',
 'kitchen_temperature',
 'bedroom_heater2_temperature',
 'kitchen_fridge_partial_energy',
 'bedroom_switch_bottom_right',
 'office_shutter',
 'bedroom_drawer2',
 'global_temperature_ext',
 'bedroom_heater1_command',
 'bathroom_heater_base_setpoint',
 'bedroom_bed_pressure',
 'livingroom_table_noise',
 'global_lighting_voltage',
 'office_switch_middle',
 'global_gas_total',
 'global_waterheater_status',
 'office_door',
 'kitchen_washingmachine_power',
 'staircase_switch_left',
 'bedroom_heater2_base_setpoint',
 'entrance_heater_effective_mode',
 'bedroom_heater1_temperature',
 'entrance_light1',
 'livingroom_shutter1',
 'livingroom_heater1_base_setpoint',
 'kitchen_CO2',
 'walkway_switch2_top_left',
 'kitchen_switch_top_left',
 'kitchen_fridge_total_energy',
 'bathroom_shower_door',
 'bedroom_heater1_base_setpoint',
 'kitchen_light1',
 'kitchen_hood_partial_energy',
 'global_waterheater_voltage',
 'kitchen_cupboard4',
 'bathroom_sink_coldwater_instantaneous',
 'bedroom_noise',
 'kitchen_humidity',
 'kitchen_oven_power',
 'kitchen_presence',
 'livingroom_luminosity',
 'E',
 'ENE',
 'ESE',
 'N',
 'NE',
 'NNE',
 'NNW',
 'NW',
 'S',
 'SE',
 'SSE',
 'SSW',
 'SW',
 'W',
 'WNW',
 'WSW',
 'down',
 'equal',
 'up',
 'Couvert',
 'brouillard',
 'brume',
 'ensoleillé',
 'légères pluies',
 'nuageux',
 'partiellement ensoleillé',
 'peu nuageux',
 'cloudy',
 'few-showers',
 'fog',
 'mostly-cloudy-day',
 'partly-cloudy-day',
 'sunny',]]

In [56]:
y = all_data[['Bathroom|Cleaning',
 'Bathroom|Showering',
 'Bathroom|Using_the_sink',
 'Bathroom|Using_the_toilet',
 'Bedroom|Cleaning',
 'Bedroom|Dressing',
 'Bedroom|Napping',
 'Bedroom|Reading',
 'Entrance|Entering',
 'Entrance|Leaving',
 'Kitchen|Cleaning',
 'Kitchen|Cooking',
 'Kitchen|Preparing',
 'Kitchen|Washing_the_dishes',
 'Living_room|Cleaning',
 'Living_room|Computing',
 'Living_room|Eating',
 'Living_room|Watching_TV',
 'Office|Cleaning',
 'Office|Computing',
 'Office|Watching_TV',
 'Staircase|Going_down',
 'Staircase|Going_up',
 'Toilet|Using_the_toilet']]

In [57]:
y = y.values

In [58]:
X = X.values

In [59]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)
x_test, x_val, y_test, y_val = train_test_split(x_test,y_test, test_size = 0.3)

#MLP Model Building

In [60]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD
# Plot
import matplotlib.pyplot as plt

In [61]:
# hyper-parameters
num_classes = 24


model = Sequential()
# first layer with 
model.add(Dense(x_train.shape[1], activation='relu', input_shape=(x_train.shape[1],)))
# second layer with 
model.add(Dense((x_train.shape[1]+num_classes)/2, activation='relu'))
# Dropout of 50%
model.add(Dropout(0.5))
# Camada de 20 neurônios
model.add(Dense((x_train.shape[1]+num_classes)/2, activation='relu'))

# Classification Layer 
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 265)               70490     
                                                                 
 dense_5 (Dense)             (None, 144)               38304     
                                                                 
 dropout_1 (Dropout)         (None, 144)               0         
                                                                 
 dense_6 (Dense)             (None, 144)               20880     
                                                                 
 dense_7 (Dense)             (None, 24)                3480      
                                                                 
Total params: 133,154
Trainable params: 133,154
Non-trainable params: 0
_________________________________________________________________


In [62]:
sgd = SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [63]:
%%time
batch_size = 34
epochs = 30
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))

Epoch 1/30
7786/7786 [==============================] - 44s 6ms/step - loss: 1.0884 - accuracy: 0.7274 - val_loss: 0.6631 - val_accuracy: 0.8590
Epoch 2/30
7786/7786 [==============================] - 41s 5ms/step - loss: 0.5894 - accuracy: 0.8499 - val_loss: 0.4283 - val_accuracy: 0.8926
Epoch 3/30
7786/7786 [==============================] - 37s 5ms/step - loss: 0.4298 - accuracy: 0.8861 - val_loss: 0.3355 - val_accuracy: 0.9071
Epoch 4/30
7786/7786 [==============================] - 40s 5ms/step - loss: 0.3530 - accuracy: 0.9007 - val_loss: 0.2874 - val_accuracy: 0.9146
Epoch 5/30
7786/7786 [==============================] - 42s 5ms/step - loss: 0.3084 - accuracy: 0.9112 - val_loss: 0.2536 - val_accuracy: 0.9229
Epoch 6/30
7786/7786 [==============================] - 39s 5ms/step - loss: 0.2771 - accuracy: 0.9193 - val_loss: 0.2293 - val_accuracy: 0.9295
Epoch 7/30
7786/7786 [==============================] - 37s 5ms/step - loss: 0.2521 - accuracy: 0.9275 - val_loss: 0.2076 - val_ac

In [80]:
%%time
from sklearn.metrics import classification_report, confusion_matrix
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07898915559053421
Test accuracy: 0.975202739238739
CPU times: user 3.25 s, sys: 181 ms, total: 3.43 s
Wall time: 5.55 s


In [81]:
%%time
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07863140106201172
Test accuracy: 0.974501371383667
CPU times: user 7.76 s, sys: 453 ms, total: 8.21 s
Wall time: 9.48 s


In [66]:
# Classification
y_pred = model.predict(x_test)

y_pred

2482/2482 [==============================] - 7s 3ms/step


array([[6.6448880e-10, 1.4111084e-13, 2.7932612e-10, ..., 5.0896096e-09,
        1.1855605e-09, 1.6046823e-08],
       [3.4578544e-09, 1.9806750e-12, 2.1395374e-09, ..., 1.6494425e-08,
        4.9692721e-09, 8.7080686e-08],
       [5.8750360e-05, 8.3069403e-07, 8.5580159e-06, ..., 1.7037909e-06,
        2.5526758e-06, 3.1897496e-06],
       ...,
       [1.7164000e-08, 8.0925044e-11, 2.1790727e-08, ..., 1.5408052e-07,
        4.3126676e-08, 6.0252347e-07],
       [2.3247321e-10, 2.2833792e-14, 9.5804704e-11, ..., 1.7774138e-09,
        5.9583166e-10, 1.5958189e-08],
       [3.5212481e-06, 5.8193923e-08, 3.5779911e-05, ..., 6.1450328e-06,
        1.4031660e-05, 7.4949683e-05]], dtype=float32)

In [67]:
for i in range(0,len(y_pred)):
# Get the index of the maximum value in the output list
  max_index = np.argmax(y_pred[i])
  # Create a binary vector with the maximum value as 1 and other values as 0
  y_pred[i] = np.zeros(len(y_pred[i]))

  y_pred[i][max_index] = 1

In [68]:
labels = ['Bathroom|Cleaning',
 'Bathroom|Showering',
 'Bathroom|Using_the_sink',
 'Bathroom|Using_the_toilet',
 'Bedroom|Cleaning',
 'Bedroom|Dressing',
 'Bedroom|Napping',
 'Bedroom|Reading',
 'Entrance|Entering',
 'Entrance|Leaving',
 'Kitchen|Cleaning',
 'Kitchen|Cooking',
 'Kitchen|Preparing',
 'Kitchen|Washing_the_dishes',
 'Living_room|Cleaning',
 'Living_room|Computing',
 'Living_room|Eating',
 'Living_room|Watching_TV',
 'Office|Cleaning',
 'Office|Computing',
 'Office|Watching_TV',
 'Staircase|Going_down',
 'Staircase|Going_up',
 'Toilet|Using_the_toilet']

In [69]:
df_pred = pd.DataFrame( y_pred,columns=labels)

In [70]:
original_array_pred = df_pred.idxmax(axis=1)

In [71]:
df_test = pd.DataFrame( y_test,columns=labels)
original_array_test = df_test.idxmax(axis=1)
original_array_test

0        Office|Computing
1        Office|Computing
2         Bedroom|Reading
3        Office|Computing
4        Office|Computing
               ...       
79411    Office|Computing
79412    Office|Computing
79413    Office|Computing
79414    Office|Computing
79415     Bedroom|Napping
Length: 79416, dtype: object

In [72]:
cm = confusion_matrix(original_array_test,original_array_pred)

In [73]:
df_cm = pd.DataFrame( cm,columns=labels, index=labels)

In [74]:
df_cm

,Bathroom|Cleaning,Bathroom|Showering,Bathroom|Using_the_sink,Bathroom|Using_the_toilet,Bedroom|Cleaning,Bedroom|Dressing,Bedroom|Napping,Bedroom|Reading,Entrance|Entering,Entrance|Leaving,...,Living_room|Cleaning,Living_room|Computing,Living_room|Eating,Living_room|Watching_TV,Office|Cleaning,Office|Computing,Office|Watching_TV,Staircase|Going_down,Staircase|Going_up,Toilet|Using_the_toilet
Bathroom|Cleaning,0,0,58,8,0,17,1,2,0,0,...,0,0,1,0,0,0,0,1,2,0
Bathroom|Showering,0,2619,0,0,0,0,1,0,2,0,...,0,0,0,4,0,0,0,2,2,1
Bathroom|Using_the_sink,0,431,468,2,0,0,1,0,0,0,...,0,4,2,0,0,0,0,0,2,0
Bathroom|Using_the_toilet,0,0,71,173,0,0,1,0,0,0,...,0,1,0,0,0,5,0,0,0,0
Bedroom|Cleaning,0,0,0,0,0,38,0,8,0,0,...,0,0,0,0,0,0,0,1,4,0
Bedroom|Dressing,0,0,0,0,0,221,15,27,0,0,...,0,10,4,0,0,0,0,0,3,0
Bedroom|Napping,0,0,0,0,0,9,2894,1,0,0,...,0,1,3,0,0,0,0,0,0,0
Bedroom|Reading,0,0,0,0,0,13,0,3214,0,0,...,0,0,0,0,0,0,0,0,0,0
Entrance|Entering,0,0,0,0,0,0,0,0,200,86,...,3,0,5,1,0,0,0,3,2,0
Entrance|Leaving,0,0,0,0,0,0,0,0,31,135,...,0,0,4,0,0,0,0,0,3,0


In [75]:
df_cm.to_csv("confusion_matrix_MLP_v2_24_classes.csv",index=True)

In [76]:
# y_true and y_pred are your true and predicted labels, respectively
report = classification_report(y_test, y_pred, target_names=labels, output_dict=True)

# Calculate the weighted average of precision, recall, and F1 score
precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1_score = report['weighted avg']['f1-score']

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
weights = np.sum(y_test, axis=0)
precision_weighted = np.sum(precision * weights) / np.sum(weights)
recall_weighted = np.sum(recall * weights) / np.sum(weights)
f1_score_weighted = np.sum(f1_score * weights) / np.sum(weights)

print("Precision:", precision_weighted)
print("Recall:", recall_weighted)
print("F1 score:", f1_score_weighted)

Precision: 0.9708565648844947
Recall: 0.9745013599274706
F1 score: 0.9712850355290832


In [78]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9745013599274706
